In [1]:
!pip install -q ultralytics lxml

import os, shutil, random, glob, hashlib
from pathlib import Path
from lxml import etree

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.6 MB/s eta 0:00:0000:01


In [2]:
from ultralytics import YOLO
print("Ultralytics OK")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics OK


In [3]:
UAV_ROOT = Path("/kaggle/input/traffic-images-captured-from-uavs")
RND_ROOT = Path("/kaggle/input/roundabout-aerial-images-for-vehicle-detection/original/original")

UAV_DATASET_DIR = UAV_ROOT / "dataset"     # aqui vimos sec9/*.txt etc.
RND_ANN_DIR = RND_ROOT / "annotations"     # aqui estan los XML

print("UAV_DATASET_DIR exists:", UAV_DATASET_DIR.exists())
print("RND_ANN_DIR exists:", RND_ANN_DIR.exists())

# Buscar donde estan las imagenes de roundabout automaticamente
# (a veces estan en 'images', 'frames', 'original', etc.)
rnd_images_candidates = [p for p in RND_ROOT.rglob("*") if p.is_dir() and p.name.lower() in ["images", "imgs", "frames"]]
print("Possible roundabout image dirs:", rnd_images_candidates[:5])

UAV_DATASET_DIR exists: True
RND_ANN_DIR exists: True
Possible roundabout image dirs: [PosixPath('/kaggle/input/roundabout-aerial-images-for-vehicle-detection/original/original/imgs')]


In [4]:
def get_xml_classes(xml_paths, max_files=200):
    classes = set()
    for i, xp in enumerate(xml_paths[:max_files]):
        tree = etree.parse(str(xp))
        for obj in tree.findall(".//object"):
            name = obj.findtext("name")
            if name:
                classes.add(name.strip())
    return sorted(classes)

xml_paths = sorted(RND_ANN_DIR.glob("*.xml"))
print("Num XML:", len(xml_paths))
print("XML classes (sample):", get_xml_classes(xml_paths, max_files=200))

Num XML: 15474
XML classes (sample): ['cycle', 'vehicle']


In [5]:
# Mapa: nombre_en_XML = indice_clase_YOLO
# Opcion simple (1 clase): todo es vehicle
CLASS_MAP = {
    "vehicle": 0,
    "car": 0,
    "cars": 0,
    "auto": 0
}

YOLO_NAMES = ["vehicle"]  # nombres finales del dataset
NC = len(YOLO_NAMES)

print("NC:", NC)
print("YOLO_NAMES:", YOLO_NAMES)

NC: 1
YOLO_NAMES: ['vehicle']


In [6]:
def voc_to_yolo(xml_path: Path, class_map: dict):
    tree = etree.parse(str(xml_path))
    root = tree.getroot()

    # Tamano imagen
    size = root.find("size")
    w = int(size.findtext("width"))
    h = int(size.findtext("height"))

    yolo_lines = []
    for obj in root.findall("object"):
        name = (obj.findtext("name") or "").strip()
        if name not in class_map:
            # Si aparece una clase inesperada, la ignoramos para no romper
            continue
        cls = class_map[name]

        b = obj.find("bndbox")
        xmin = float(b.findtext("xmin"))
        ymin = float(b.findtext("ymin"))
        xmax = float(b.findtext("xmax"))
        ymax = float(b.findtext("ymax"))

        # Convertir a YOLO normalizado
        cx = ((xmin + xmax) / 2.0) / w
        cy = ((ymin + ymax) / 2.0) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h

        # Clip por seguridad
        cx = min(max(cx, 0.0), 1.0)
        cy = min(max(cy, 0.0), 1.0)
        bw = min(max(bw, 0.0), 1.0)
        bh = min(max(bh, 0.0), 1.0)

        yolo_lines.append(f"{cls} {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}")

    # nombre archivo (sin extension) para emparejar con imagen
    filename = (root.findtext("filename") or xml_path.stem).strip()
    stem = Path(filename).stem

    return stem, yolo_lines

# Test rapido con 1 xml
stem_test, lines_test = voc_to_yolo(xml_paths[0], CLASS_MAP)
print("Example stem:", stem_test)
print("Example lines:", lines_test[:3])

Example stem: 00001_frame000000_original
Example lines: ['0 0.719010 0.124074 0.022396 0.083333', '0 0.943229 0.133333 0.039583 0.037037', '0 0.787240 0.153241 0.042188 0.041667']


In [9]:
OUT_ROOT = Path("/kaggle/working/mye_dataset")
for p in [
    OUT_ROOT / "images/train", OUT_ROOT / "images/val",
    OUT_ROOT / "labels/train", OUT_ROOT / "labels/val"
]:
    p.mkdir(parents=True, exist_ok=True)

def sha1_path(p: Path) -> str:
    return hashlib.sha1(str(p).encode("utf-8")).hexdigest()[:10]


In [10]:
# Index de imagenes de roundabout por stem
IMG_EXTS = [".jpg", ".jpeg", ".png"]
rnd_all_images = []
for ext in IMG_EXTS:
    rnd_all_images += list(RND_ROOT.rglob(f"*{ext}"))

rnd_img_by_stem = {}
for ip in rnd_all_images:
    rnd_img_by_stem[ip.stem] = ip

print("Roundabout images found:", len(rnd_all_images))

roundabout_samples = []
skipped = 0

for xp in xml_paths:
    stem, yolo_lines = voc_to_yolo(xp, CLASS_MAP)
    if stem not in rnd_img_by_stem:
        skipped += 1
        continue
    img_path = rnd_img_by_stem[stem]
    # Guardamos las lineas YOLO en memoria por ahora
    roundabout_samples.append((img_path, yolo_lines, f"rnd_{sha1_path(xp)}_{stem}"))

print("Roundabout samples:", len(roundabout_samples))
print("Roundabout skipped (no image match):", skipped)
print("Example sample:", roundabout_samples[0][0], "lines:", len(roundabout_samples[0][1]))


Roundabout images found: 15474
Roundabout samples: 15474
Roundabout skipped (no image match): 0
Example sample: /kaggle/input/roundabout-aerial-images-for-vehicle-detection/original/original/imgs/00001_frame000000_original.jpg lines: 12


In [11]:
uav_label_paths = sorted(UAV_DATASET_DIR.rglob("*.txt"))
print("UAV labels:", len(uav_label_paths))

def find_matching_image_for_label(label_path: Path):
    # probar mismas rutas con extensiones comunes
    for ext in IMG_EXTS:
        cand = label_path.with_suffix(ext)
        if cand.exists():
            return cand
    return None

uav_samples = []
skipped_uav = 0

for lp in uav_label_paths:
    imgp = find_matching_image_for_label(lp)
    if imgp is None:
        skipped_uav += 1
        continue
    # nombre unico para evitar colisiones
    uav_samples.append((imgp, lp, f"uav_{sha1_path(lp)}_{lp.stem}"))

print("UAV samples:", len(uav_samples))
print("UAV skipped (no image match):", skipped_uav)
print("Example UAV:", uav_samples[0][0], uav_samples[0][1])


UAV labels: 16367
UAV samples: 16367
UAV skipped (no image match): 0
Example UAV: /kaggle/input/traffic-images-captured-from-uavs/dataset/sec1/sec1_frame_000001.png /kaggle/input/traffic-images-captured-from-uavs/dataset/sec1/sec1_frame_000001.txt


In [15]:
SEED = 42
random.seed(SEED)

all_samples = []

# Roundabout: (img_path, yolo_lines, new_stem)
for imgp, lines, newstem in roundabout_samples:
    all_samples.append(("roundabout", imgp, lines, newstem))

# UAV: (img_path, label_path, new_stem)
for imgp, labelp, newstem in uav_samples:
    all_samples.append(("uav", imgp, labelp, newstem))

random.shuffle(all_samples)

val_ratio = 0.1
n_val = int(len(all_samples) * val_ratio)
val_set = all_samples[:n_val]
train_set = all_samples[n_val:]

print("Total samples:", len(all_samples))
print("Train:", len(train_set), "Val:", len(val_set))

def link_or_copy(src, dst):
    """
    Intenta crear un enlace simbólico (no ocupa espacio).
    Si no es posible, copia el archivo.
    """
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists():
        return
    try:
        os.symlink(src, dst)
    except OSError:
        shutil.copy2(src, dst)

def write_label(lines_or_path, dst_lbl, src_type):
    """
    Escribe o enlaza el label según el tipo de dataset.
    """
    dst_lbl.parent.mkdir(parents=True, exist_ok=True)
    if src_type == "roundabout":
        lines = lines_or_path
        dst_lbl.write_text(
            "\n".join(lines) + ("\n" if lines else ""),
            encoding="utf-8"
        )
    else:
        # UAV: label ya es YOLO, lo enlazamos
        link_or_copy(lines_or_path, dst_lbl)

def copy_sample(sample, split):
    """
    Crea una muestra en el dataset final SIN copiar imágenes grandes.
    """
    src_type, imgp, labinfo, newstem = sample

    # Imagen → ENLACE simbólico
    dst_img = OUT_ROOT / f"images/{split}" / (newstem + imgp.suffix.lower())
    link_or_copy(imgp, dst_img)

    # Label → escribir (roundabout) o enlazar (UAV)
    dst_lbl = OUT_ROOT / f"labels/{split}" / (newstem + ".txt")
    write_label(labinfo, dst_lbl, src_type)

for s in train_set:
    copy_sample(s, "train")
for s in val_set:
    copy_sample(s, "val")

print("Done copying.")
print("Train images:", len(list((OUT_ROOT/'images/train').glob('*'))))
print("Val images:", len(list((OUT_ROOT/'images/val').glob('*'))))


Total samples: 31841
Train: 28657 Val: 3184
Done copying.
Train images: 28657
Val images: 3184


In [16]:
!du -sh /kaggle/working/mye_dataset

250M	/kaggle/working/mye_dataset


In [17]:
!ls /kaggle/working/mye_dataset/images/train | head

rnd_0002066fe6_00038_frame000040_original.jpg
rnd_0003f0799b_00001_frame000314_original.jpg
rnd_0005875c07_00050_frame000388_original.jpg
rnd_00089c084c_00001_frame000028_original.jpg
rnd_00095a90a1_00050_frame000079_original.jpg
rnd_000eb3c0ac_00050_frame002210_original.jpg
rnd_001462c253_00001_frame001767_original.jpg
rnd_0015bc497b_00053_frame001068_original.jpg
rnd_001f2072b0_00053_frame001197_original.jpg
rnd_0028699508_00015_frame000051_original.jpg
ls: write error: Broken pipe


In [18]:
data_yaml_path = OUT_ROOT / "data.yaml"
data_yaml_path.write_text(
    "\n".join([
        f"path: {OUT_ROOT}",
        "train: images/train",
        "val: images/val",
        f"nc: {NC}",
        f"names: {YOLO_NAMES}",
        ""
    ]),
    encoding="utf-8"
)

print(data_yaml_path.read_text())


path: /kaggle/working/mye_dataset
train: images/train
val: images/val
nc: 1
names: ['vehicle']



In [22]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # ligero y rapido

results = model.train(
    data=str(data_yaml_path),
    imgsz=960,
    epochs=25,
    batch=16,
    device=0,
    project="/kaggle/working/runs_mye",
    name="train_mye",
    verbose=True
)


Ultralytics 8.4.12 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/mye_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train_mye4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pati

In [23]:
best_path = Path("/kaggle/working/runs_mye/train_mye/weights/best.pt")
print("Best exists:", best_path.exists(), best_path)

!cp /kaggle/working/runs_mye/train_mye/weights/best.pt /kaggle/working/MyE_best.pt
!ls -lah /kaggle/working | grep MyE_best.pt


Best exists: True /kaggle/working/runs_mye/train_mye/weights/best.pt
-rw-r--r-- 1 root root  18M Feb  7 21:28 MyE_best.pt
